In [1]:
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.keys import Keys
from time import sleep
import sys
import re
from collections import defaultdict
import pandas as pd
import datetime as dt
from datetime import timedelta
from hashlib import sha256
import os.path
from src import DATA_DIR
from hazm import Normalizer, sent_tokenize, word_tokenize
import plotly.express as px
from collections import Counter
from src import DATA_DIR
import pandas as pd
from itertools import groupby
from collections import Counter
from hazm import Normalizer, sent_tokenize, word_tokenize
from hashlib import sha256
import copy
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

In [6]:
class Twitter_scraper:
    def __init__(self, driver):
        self.driver = driver
        

    def save_file(self, list_, exprot_file_name):
        df = pd.DataFrame(list_, columns=['Search_Query', 'Tweet_ID',
                                          'Username', 'User_ID',
                                          'DateTime', 'Text',
                                          'Reply_to', 'Hashtag',
                                          'Reply', 'Retweet',
                                          'Like',
                                         ])

        if not os.path.isfile(DATA_DIR / f'data/{exprot_file_name}.csv'):
            df.to_csv(DATA_DIR / f'data/{exprot_file_name}.csv', encoding="utf-8-sig", index=False)

        else:
            df.to_csv(DATA_DIR / f'data/{exprot_file_name}.csv', encoding="utf-8-sig", mode='a', header=False, index=False)
            
            
    
    
    def create_search_query(self, search_query:dict):
        
        '''
        *** Create Search Query ***
        :param search_query: (dict or path) Gets search query. If search_querys type is dictionary, It can have following keys:
            - include: (list) Words that must be in tweet text. Each element is a list. This method will combine different elements with 'AND'
              operator and each element words with 'OR' operator.
              
            - not_include: (list) Words that shouldn't be in tweet text. Each element is a list. This method will combine different elements
              with 'AND' operator and each element words with 'OR' operator.

            - from: (list) Filters tweets that sent from specific Twitter accounts

            - to: (list) Filters tweets that sent in reply to specific Twitter accounts

            - since: (str, samp = “2015-12-21”) Containing tweets that sent since a specific date

            - until: (str, samp = “2022-12-21”) Containing tweets that sent until a specific date

            - mentioning: (list) Containing tweets that mentioning specific Twitter accounts

            - question: ('True', 'False') If True, gets all tweet, that were recognized by twitter AI as a question
                
        '''

        search_text = ''
        _ = []

        for elem in search_query['include']:
            _.append("((" + ") OR (".join(elem) + "))")
        search_text = "((" + ") AND (".join(_) + "))"

        if 'not_include' in search_query.keys():
            _ = []
            for elem in search_query['not_include']:
                _.append("((" + ") AND (".join(elem) + "))")
            search_text += " -((" + ") OR (".join(_) + "))"

        if 'from' in search_query.keys():
            search_text += ' (from:' + ' OR from:'.join(search_query['from']) + ')'

        if 'to' in search_query.keys():
            search_text += ' (to:' + ' OR to:'.join(search_query['to']) + ')'

        if 'lang' in search_query.keys():
            search_text += ' lang:' + search_query['lang']

        if 'since' in search_query.keys():
            search_text += f" since:{search_query['since']}"

        if 'until' in search_query.keys():
            search_text += f" until:{search_query['until']}"

        if 'mentioning' in search_query.keys():
            search_text += ' (@' + ' @'.join([search_query['mentioning']]) + ')'
            
        return search_text
    
    
    
    def get_tweet_data(self, exprot_file_name, search_query, max_tweet=None):
        
        ''' 
        *** Extract tweets ***
        :param search_query: (dict or path)
            - If search_query is a path, it should redirect to a .txt file that contains search query you want
            - If search_query is a dict. it should defined like 'create_search_query' method.
        
        :param max_tweet: (int) Ahen reachs to the number, running will stop
        '''
        
        sleep(1)
        self.driver.get('https://twitter.com/explore')
        sleep(4)
        
        
        # Read Unique IDs
        unique_tweet_ids = set()
        if os.path.isfile(DATA_DIR / f'data/{exprot_file_name}.csv'):
            df = pd.read_csv(DATA_DIR / f'data/{exprot_file_name}.csv', encoding="utf-8-sig")
            unique_tweet_ids = set(df.Tweet_ID)
        
        
        # Create Search Query
        search_list = []
        if isinstance(search_query, dict):
            search_list.append(self.create_search_query(search_query))
        else:
            with open(search_query) as fp:
                search_list = fp.read().split('\n')
        
        progress = 1
        for exp_ind, search_text in enumerate(search_list):
            # Enter Search Query in Twitter search box
            sleep(4)
            search = self.driver.find_element('xpath', '//label[@data-testid="SearchBox_Search_Input_label"]')
            search.send_keys(Keys.CONTROL + "a")
            search.send_keys(Keys.DELETE)
            search.send_keys(search_text)
            search.send_keys(Keys.RETURN)

            # Go to 'Latest' tweets
            sleep(10)
            self.driver.find_element('link text', 'Latest').click()

            # Scraping
            scrolling = True
            list_ = []
            j = 1

            while scrolling:
                sleep(2)
                tweets = self.driver.find_elements('xpath', '//article[@data-testid="tweet"]')
                sleep(1)
                
                if progress >= 500 * j:
                    self.driver.close()
                    sleep(15)
                    options = EdgeOptions()
                    options.use_chromium = True
                    options.add_argument("-inprivate")
                    options.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
                    self.driver = Edge('/mnt/f/msedgedriver.exe', options=options)
                    sleep(3)

                    self.driver.get('https://twitter.com/explore')
                    sleep(4)
                    search = self.driver.find_element('xpath', '//label[@data-testid="SearchBox_Search_Input_label"]')
                    search.send_keys(Keys.CONTROL + "a")
                    search.send_keys(Keys.DELETE)
                    if j > 1:
                        search_text = search_text[:-17]
                    search.send_keys(search_text + ' until:' + str(datetime)[:10])
                    search.send_keys(Keys.RETURN)
                    sleep(6)
                    self.driver.find_element('link text', 'Latest').click()
                    sleep(2)
                    
                    j += 1
                
                for tweet in tweets[-min(24, len(tweets)-1):]:
                    try:
                        reply_to, hashtag = str(), str()
                        text = tweet.find_element('xpath', './/div[@data-testid="tweetText"]').text

                        if 'Replying to \n@' in tweet.text:
                            reply_to = tweet.find_element(
                                'xpath', './/div[@class="css-901oao r-1bwzh9t r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-qvutc0"]'
                            ).text.split()
                            reply_to = [id_ for id_ in reply_to if id_[0] == '@']

                        if '#' in text:
                            words = text.split()
                            hashtag = [word for word in words if word[0] == '#']

                        reply = tweet.find_element('xpath', './/div[@data-testid="reply"]').text
                        retweet = tweet.find_element('xpath', './/div[@data-testid="retweet"]').text
                        like = tweet.find_element('xpath', './/div[@data-testid="like"]').text

                        header = tweet.find_element('xpath', './/div[@data-testid="User-Names"]')
                        _ = header.find_element('xpath', './/time').get_attribute('datetime')
                        datetime = dt.datetime.strptime(_, '%Y-%m-%dT%H:%M:%S.000Z') + timedelta(hours=4.5)

                        info = header.text.split('@')
                        username = info[0].strip('\n')
                        user_id = '@' + info[1].split()[0]

                        tweet_id = sha256("".join([user_id, str(datetime)]).encode()).hexdigest()


                        if tweet_id not in unique_tweet_ids:
                            sys.stdout.write('\r')
                            sys.stdout.write(f"Saved: {progress - progress % 100}       " +
                                             f"Current_State: [ Search_Query: {exp_ind + 1} from {len(search_list)}    " +
                                             f"Tweet: {progress}    " +
                                             f"Date: {str(datetime)[:7]} ] {' '*20}‌")
                            
                            sys.stdout.flush()
                            progress += 1

                            list_.append([exp_ind, tweet_id, username, user_id, datetime, text, reply_to, hashtag, reply, retweet, like])
                            unique_tweet_ids.add(tweet_id)

                            if len(list_) >= 100:
                                self.save_file(list_, exprot_file_name)
                                list_ = []

                            if max_tweet:
                                if progress > max_tweet:
                                    self.save_file(list_, exprot_file_name)
                                    list_ = []
                                    scrolling = False
                                    break

                    except:
                        pass

                # Scrolling
                scroll_attemp = 0
                scroll_up = 0
                
                last_positon = self.driver.execute_script('return window.pageYOffset;')

                while True:
                    self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                    sleep(4)
                    current_positon = self.driver.execute_script('return window.pageYOffset;')

                    if current_positon == last_positon:
                        scroll_attemp += 1
                        
                        if scroll_attemp == 2 and scroll_up <= 2:
                            self.driver.execute_script(f"window.scrollTo(0, {self.driver.execute_script('return window.pageYOffset;')-1800})")
                            sleep(3)
                            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                            sleep(4)
                        
                        if scroll_attemp >= 5:
                            if len(list_) > 0:
                                self.save_file(list_, exprot_file_name)
                                list_ = []

                            scrolling = False
                            break
                        else:
                            sleep(5)
                    else:
                        break

In [2]:
class Gethi_Chart:
    def __init__(self):
        pass
    
    
    def replace_compound_words(self, input_, compound_words_path, mode=1):
        with open(compound_words_path) as fp:
            compound_words = fp.read()
        
        compound_words = compound_words.split('\n')
        compound_words = {item:'_'.join(item.split()) for item in compound_words}
        
        if mode == 1:
            for key, value in compound_words.items():
                input_ = input_.str.replace(key, value)
            
        else:
            for key, value in compound_words.items():
                input_ = input_.str.replace(value, key)
                
        return input_
    
    
    def create_node_edge(self, tweet_data_dir, theme_dir, output_dir):
        df = pd.read_csv(tweet_data_dir)
        df = df.drop_duplicates(subset = "Tweet_ID")
        tweet_texts = ' Oadf6dsaf8dfa '.join(list(df.Text))
        tweet_texts2 = copy.deepcopy(tweet_texts)

        _ = pd.read_csv(theme_dir)
        themes = {_.iloc[i, 0]:[sha256(_.iloc[i, 1].encode()).hexdigest()[:18], _.iloc[i, 1].replace(" ", "_")] for i in range(len(_))}

        
        # Detect Nodes
        keys = list(themes.keys())
        keys = sorted(keys, key=len, reverse=True)

        for item in keys:
            tweet_texts = tweet_texts.replace(item, themes[item][0])

        for theme_ in themes.values():
            tweet_texts = tweet_texts.replace(theme_[0], theme_[1])

        filt = [*[item[1] for item in list(themes.values())], 'Oadf6dsaf8dfa']

        _ = list(filter(lambda x: x in filt, tweet_texts.split()))
        tweet_list = [list(group) for k, group in groupby(_, lambda x: x == "Oadf6dsaf8dfa") if not k]
        
        _ = [(item[0], item[0].replace("_", " "), item[1]) for item in Counter(_).most_common()[1:]]
        df_output_nodes = pd.DataFrame(_, columns=['Id', 'Label', 'Weight'])
        df_output_nodes.to_csv(output_dir / 'nodes.csv', index=False, encoding="utf-8-sig")
        
        
        # Detect Edges
        edges = defaultdict(int)
        for tweet in tweet_list:
            set_ = set()
            if len(tweet) <=1:
                continue

            for i in range(len(tweet)):
                for j in range(i+1, len(tweet)):
                    if tweet[i] != tweet[j]:
                        if (tweet[i], tweet[j]) in set_:
                            continue
                        else:
                            edges[(tweet[i], tweet[j])] += 1
                            set_.add((tweet[i], tweet[j]))
            
        tweet_list = []
        tweet_list_word = []

        for key, value in edges.items():
            tweet_list.append((*key, value))
            
        df_output_edges = pd.DataFrame(tweet_list, columns=['Source', 'Target', 'Weight'])
        df_output_edges.to_csv(output_dir / 'edges.csv', index=False, encoding="utf-8-sig")
        
    
    def remove_stopwords(self, text: str, stopwords: list):
        """
        :param text: text you want to delete stopwords from dat
        :param stopwords: list of stopwords
        """
        tokens = word_tokenize(text)
        tokens = filter(lambda word: word not in stopwords, tokens)
        return " ".join(tokens)


    def most_common_words(self, tweet_data_dir, stopwords_dir, output_dir):
        df = pd.read_csv(tweet_data_dir)
        df = df.drop_duplicates(subset = "Tweet_ID")
        
        df.Text = self.replace_compound_words(df.Text, DATA_DIR / 'data/Input/compound_words.txt')
        
        with open(stopwords_dir) as fp:
            stopwords = fp.read()
        stopwords = stopwords.split('\n')
        
        list_ = list(df.Text)
        list_ = [str(text) for text in list_]
        text = ' '.join(list_)
        text = self.remove_stopwords(text, stopwords)
        text = text.split()
        df = pd.DataFrame(Counter(text).most_common(), columns=['word', 'count'])
        df.word = self.replace_compound_words(df.word, DATA_DIR / 'data/Input/compound_words.txt', mode=2)
        df.to_csv(output_dir / 'most_common_words.csv', index=False, encoding="utf-8-sig")

### 
## Scraping Twitter

In [44]:
options = EdgeOptions()
options.use_chromium = True
options.add_argument("-inprivate")
options.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
driver = Edge('/mnt/f/msedgedriver.exe', options=options)

tweet = Twitter_scraper(driver)


tweet.get_tweet_data(
    search_query={'include': [[
                               'اومارکت',
                               'اوکالا',
                              ],                             
                             ],

                  'lang': 'fa',
                 },

    max_tweet=None,
    exprot_file_name='Dataset')

Exception ignored in: <function Service.__del__ at 0x7fcc1d273280>
Traceback (most recent call last):
  File "/home/kourosh/anaconda3/envs/py38/lib/python3.8/site-packages/selenium/webdriver/common/service.py", line 171, in __del__
    self.stop()
  File "/home/kourosh/anaconda3/envs/py38/lib/python3.8/site-packages/selenium/webdriver/common/service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "/home/kourosh/anaconda3/envs/py38/lib/python3.8/site-packages/selenium/webdriver/common/service.py", line 122, in send_remote_shutdown_command
    request.urlopen("%s/shutdown" % self.service_url)
  File "/home/kourosh/anaconda3/envs/py38/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/home/kourosh/anaconda3/envs/py38/lib/python3.8/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/kourosh/anaconda3/envs/py38/lib/python3.8/urllib/request.py", line 542, in _open
    result = s

### 
## Output Analysis

In [3]:
chart = Gethi_Chart()

chart.most_common_words(tweet_data_dir = DATA_DIR / 'data/Dataset.csv',
                        stopwords_dir = DATA_DIR / 'data/Input/persian_stop_words.txt',
                        output_dir = DATA_DIR / 'data/Themes'
                       )

chart.create_node_edge(tweet_data_dir = DATA_DIR / 'data/Dataset.csv',
                       theme_dir = DATA_DIR / 'data/Themes/Themes.csv',
                       output_dir = DATA_DIR / 'data/Gephi',
                      )